In [2]:
import tensorflow as tf

In [3]:
!nvidia-smi

Fri Aug  5 16:26:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   35C    P8    24W / 260W |    310MiB / 11019MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
# sys.path.insert(0,'/home/jk21/Documents/hmd/jk_classifier/lucashnegri-peakutils-51a679cd8428')
# sys.path.insert(0,'/home/jk21/Documents/hmd/jk_classifier/S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *
from keras.preprocessing import sequence
# import peakutils
from scipy import special
import scipy.io as sio
from keras.preprocessing.sequence import pad_sequences
sys.path.insert(0,'/home/jk21/Documents/hmd/jk_classifier/lucashnegri-peakutils-51a679cd8428')
import peakutils

2022-08-05 16:26:38.706652: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 16:26:38.710488: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 16:26:38.710816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 16:26:38.711270: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
pwd

'/home/jk21/Documents/hmd/jk_classifier'

In [8]:
root_dir = 'physionet.org/files/circor-heart-sound/1.0.3'
# training_data_file = root_dir + '/' + 'training_data.csv'
# training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [9]:
# data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/jk21/Downloads/Data/data/murmur/train'
test_folder = '/home/jk21/Downloads/Data/data/murmur/test'


In [10]:
model_folder = 'lcnn2'

In [11]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint

In [12]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

In [29]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [30]:
params_feature = {'samp_sec': 30,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': 128,
                  'win_length':256,
                  'n_mels': 100,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10,
                  'maxlen1': 120000,
                  'min_dist':500,
}

In [38]:
def get_features_3lb_all(data_folder, patient_files_trn, 
                          samp_sec=20, pre_emphasis = 0, hop_length=256, win_length = 512, n_mels = 100,
                          filter_scale = 1, n_bins = 80, fmin = 10, trim = 4000,maxlen1=120000,min_dist=1000,
                         use_mel = True, use_cqt = False, use_stft= False, use_raw = False,use_interval = False,use_wav2=False
                         ) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    features['cqt1'] = []
    features['stft1'] = []
    features['raw1'] = []
    features['interval'] = []
    features['wav2']=[]
#    labels = []
    features['mm_labels'] = []
    features['out_labels'] = []
    tmp_interval=[]
    tmp_wav=[]
    interval_len=[]

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in tqdm.tqdm(range(num_patient_files)):
        

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            if use_mel :
                mel1 = feature_extract_melspec(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                               win_length = win_length, n_mels = n_mels, trim = trim)[0]
            else :
                mel1 = np.zeros( (1,1,1) )
            features['mel1'].append(np.array(mel1))

            if use_cqt :
                mel2 = feature_extract_cqt(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                           n_bins = n_bins, fmin = fmin, trim = trim)[0]
            else :
                mel2 = np.zeros( (1,1,1))                
            features['cqt1'].append(np.array(mel2))

            if use_stft :
                mel3 = feature_extract_stft(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                            win_length = win_length, trim = trim)[0]
            else :
                mel3 = np.zeros( (1,1,1) )
            features['stft1'].append(np.array(mel3))

            if use_raw :
                recording1,frequency1 = librosa.load(filename)
            else :
                recording1 = np.zeros( (1) )
            features['raw1'].append(recording1)
            
            if use_wav2:
                recording1,frequency1 = librosa.load(filename)
            else :
                recording1 = np.zeros( (1) )
            tmp_wav.append(recording1)                
                
            
            if use_interval :
                               
                datos=sp.io.wavfile.read(filename)
                filtros=sio.loadmat('/home/jk21/Documents/hmd/jk_classifier/S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python/Filters1')
                
                try:
                    
                    X=datos[1]
                    X=X[trim*3:-trim*3]
                    Fs=datos[0]
            
                    Fpa20=filtros['Fpa20'];			        # High pass filter
                    Fpa20=Fpa20[0];					# High pass filter
                    Fpb100=filtros['Fpb100'];		        # Low-pass Filter
                    Fpb100=Fpb100[0];				# Low-pass Filter
            
                    Xf=FpassBand(X,Fpa20,Fpb100); 	                # Apply a passband filter
                    Xf=vec_nor(Xf);			
            
                    # Derivate of the Signal
                    dX=derivate(Xf);				# Derivate of the signal
                    dX=vec_nor(dX);					# Vector Normalizing
                    # Square of the signal
                    dy=np.square(Xf);
                    dy=vec_nor(dy);
                    
                    size=np.shape(Xf)				# Rank or dimension of the array
                    fil=size[0];					# Number of rows

                    positive=np.zeros((1,fil+1));                   # Initializating Positives Values Vector 
                    positive=positive[0];                           # Getting the Vector

                    points=np.zeros((1,fil));                       # Initializating the all Peak Points Vector
                    points=points[0];                               # Getting the point vector

                    peaks=np.zeros((1,fil));                        # Initializating the s1-s1 Peak Vector
                    peaks=peaks[0];                                 # Getting the point vector

            
                    for i in range(0,fil):
                        if dX[i]>0:
                            positive[i]=1;
                        else:
                            positive[i]=0;

                    for i in range(0,fil):
                        if (positive[i]==1 and positive[i+1]==0):
                            points[i]=Xf[i];
                        else:
                            points[i]=0;

                    indexes=peakutils.indexes(points,thres=0.5/max(points), min_dist=1000);
                    lenght=np.shape(indexes)			# Get the length of the index vector		
                    lenght=lenght[0];				# Get the value of the index vector

                    for i in range(0,lenght):
                        p=indexes[i];
                        peaks[p]=points[p];
        
                    n=np.arange(0,fil);                            # Vector to the X axes (Number of Samples)
            
                    tmp_peaks = np.diff(indexes)
                    
                    
                    tmp_interval.append(tmp_peaks)
                    
           
                except:
                    print(filename)
                    tmp_peaks = np.zeros(maxlen)
                    tmp_interval.append(tmp_peaks)
                    
            else :
                        
                tmp_peaks = np.zeros(maxlen)
                tmp_interval.append(tmp_peaks)
                
      
            
            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            ## simple impute
            if math.isnan(height) :
                height = 110.846
            if math.isnan(weight) :
                weight = 23.767
                
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels 
            mm_label = get_murmur(current_patient_data)
            out_label = get_outcome(current_patient_data)
            current_mm_labels = np.zeros(2)
            current_out_labels = np.zeros(2)
            if mm_label == 'Absent' :
                current_mm_labels = np.array([0, 0, 1])
            elif mm_label == 'unknown' :
                current_mm_labels = np.array([0, 1, 0])
            else :
                mm_loc = get_murmur_loc(current_patient_data)
                if mm_loc == 'nan' :
                    current_mm_labels = np.array([0.9, 0.05, 0.05])
                else :
                    mm_loc = mm_loc.split('+')
                    if locations in mm_loc :
                        current_mm_labels = np.array([1, 0, 0])
                    else :
                        current_mm_labels = np.array([0.7, 0.2, 0.1])

            if out_label == 'Normal' :
                current_out_labels = np.array([0, 1])
            else :
                current_out_labels = np.array([1, 0])
#                if mm_label == 'Absent' :
#                    current_out_labels = np.array([0.8, 0.2])
#                elif mm_label == 'unknown' :
#                    current_out_labels = np.array([0.85, 0.15])
#                else :
#                    current_out_labels = np.array([1, 0])
                
            features['mm_labels'].append(current_mm_labels)
            features['out_labels'].append(current_out_labels)

    if use_mel :
        M, N = features['mel1'][0].shape
        for i in range(len(features['mel1'])) :
            
            features['mel1'][i] = features['mel1'][i].reshape(M,N,1)
            
    else :
        M, N, _ = features['mel1'][0].shape
    print("melspec: ", M,N)
    
    mel_input_shape = (M,N,1)
        
    if use_cqt :
        M, N= features['cqt1'][0].shape
        for i in range(len(features['cqt1'])) :
            features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)
       
    else :
        M, N,__ = features['cqt1'][0].shape
    print("cqt: ", M,N)
    cqt_input_shape = (M,N,1)

    
    if use_stft :
        M, N = features['stft1'][0].shape
        for i in range(len(features['stft1'])) :
            features['stft1'][i] = features['stft1'][i].reshape(M,N,1)
        
    else :
        M, N ,__= features['stft1'][0].shape
    print("stft: ", M,N)
    stft_input_shape = (M,N,1)
    
    if use_interval:
        
        for i in range(len(tmp_interval)):
            tmp_len = len(tmp_interval[i])
            interval_len.append(tmp_len)
        
        
        interval_len = np.array(interval_len)
        max_interval_len = np.max(interval_len)
        
        padded =pad_sequences(tmp_interval, maxlen=max_interval_len, dtype='float64', padding='post', truncating='post', value=0.0)
        
        for i in range(len(padded)):
            features['interval'].append(padded[i])
        
    else:
        for i in range(len(tmp_interval)):
            features['interval'].append(tmp_interval[i])

    if use_interval:
        
        for i in range(len(features['interval'])):
            features['interval'][i]= features['interval'][i].reshape(-1,1)
        
    else:
        for i in range(len(features['interval'])):
            features['interval'][i]= features['interval'][i].reshape(-1,1)

    for k1 in features.keys() :
        features[k1] = np.array(features[k1])          

    
    if use_wav2:
        padded =pad_sequences(tmp_wav, maxlen=maxlen1, dtype='float64', padding='pre', truncating='pre', value=0.0)
        tmp_pad = padded[:,np.newaxis,:]
        
        tmp=[]
        for i in range(len(tmp_pad)):
            tmp_feature = model(tmp_pad[i])
            tmp.append(tmp_feature)
            
        tmp=np.array(tmp, dtype=np.float32)
        new_tmp1=tmp.reshape(-1,374,32)
        features['wav2']=new_tmp1
        
    
    interval_input_shape = features['interval'].shape[1:]
    
    M,N = interval_input_shape
    
    print("interval: ", M,N)
    
    wav2_input_shape = features['wav2'].shape[1:]
    
    M,N = wav2_input_shape
    
    print("wav2: ", M,N)
    
    return features, mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape,wav2_input_shape

In [39]:
features_trn,mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape,wav2_input_shape = get_features_3lb_all(train_folder, patient_files_trn[:20], **params_feature,use_mel = True, use_cqt = False, use_stft = False,use_raw=False,use_interval=True,use_wav2=True)

100%|███████████████████████████████████████████| 20/20 [01:08<00:00,  3.42s/it]


melspec:  100 938
cqt:  1 1
stft:  1 1
interval:  52 1
wav2:  374 32
